Programa que, dado conjuntos de las mismas observaciones arrugadas a nbins=32,64,128,256,512
calcular sus TOAs y, a partir de ellos, calcula sus residuos. Adicionalmente, grafica el valor
de la RMS correspondiente a cada conjunto junto a sus respectivas barras de error.

In [ ]:
#-----------------------------------------
# importamos los paquetes que vamos a usar
#-----------------------------------------

import os
import shutil
import numpy as np
import glob                         # para hacer listas de archivo
import subprocess                   # para usar subprocesos

In [31]:
ant = 'A1'                                         # antena usada (A1 o A2)    
obs_template = np.loadtxt( './templates/obs_template_' + ant + '.dat' , skiprows=0 , dtype=str , usecols=(0) )  # observación originalmente usadada como template

In [32]:
#-----------------------------------------------
# generamos carpetas para guardar los resultados
#-----------------------------------------------

dirName = "./results_nbins_" + ant            # nombre de la carpeta en que guardaremos los resultados
if not os.path.exists(dirName):
    os.mkdir(dirName)

    
os.chdir(dirName + '/')                       # entramos en la carpeta en la cual guadaremos los resultados
    
if not os.path.exists('./timing_files'):
    os.mkdir('./timing_files')                # creamos la carpeta que contendrá los archivos .tim       
    
os.chdir('..')                                # salimos de la carpeta en la cual guadaremos los resultados
    
    
shutil.copy('./J0437-4715.par', './results_nbins_' + ant + '/timing_files/')     # copiamos el archivo .par a la nueva carpeta para poder usar tempo2

# Estos son arreglos que usaremos para graficar

nbin_array = np.zeros(5)
rms = np.zeros(5)

'./results_nbins_A1/timing_files/J0437-4715.par'

In [34]:
#--------------------------------------------------------
# Ahora trabajamos con los .pfds de nbin=1024,512,256,...
#--------------------------------------------------------

i = 0
for nbin in [64, 128, 256, 512, 1024]:
    
    nbin_array[i] = nbin

    folder = './' + ant + '_' + str(nbin) + '/'                # nombre de la carpeta que contiene los .pfds
    
    template = 'template_' + str(nbin) + '_' + ant + '.std.sm' # nombre del template
    
    shutil.copy('./templates/' + template, folder)             # copiamos el template a la carpeta

    
    os.chdir(folder)                            # entramos al directorio que contiene los .pfds
    pfd_files = glob.glob('*pfd')               # leemos los .pfds
    

#-----------------------------------------
# generamos los TOAs (el archivo .tim)
#-----------------------------------------

# primero, generamos un string que contenga todos los archivos .pfd menos el que usamos como template

    pfd_files.remove(str(obs_template))     # sacamos de la lista de observaciones a la que usamos como template

    archivos = ''
    for pfd in pfd_files:
        archivos = archivos + pfd + ' '     # hacemos un string que contenga al resto de las observaciones

# ahora llamamos a pat para generar el archivo .tim

    timing_file = "timing_" + str(nbin) + ".tim"       # nombre del archivo .tim

    subprocess.check_call( "pat -FT -A PGS -s " + template + " -f >" + timing_file + " 'tempo2' " + archivos, shell = True)
    
    os.chdir('..')                                     # salimos del directorio que contiene los .pfds
    
    shutil.copy(folder + '/' + timing_file, "./results_nbins_" + ant + '/timing_files/')             # copiamos el archivo .tim a la carpeta
    
    # si las observaciones son de A1, hay que hacer una corrección por la frecuencia

    if ant == 'A1':
        subprocess.check_call(["sed -i -e 's/1414.8750/1414.4375/g' ./results_nbins_" + ant + "/timing_files/" + timing_file], shell=True)
        
#-----------------------------------------
# calculamos el rms usando tempo2
#-----------------------------------------

    residue_file = "residues_" + str(nbin) + ".dat"   # nombre del archivo de los residuos
    
    timing_folder = './results_nbins_' + ant + 'timing_files/'

    subprocess.check_call( "tempo2 >" + residue_file + " -us -f J0437-4715.par " + timing_file, cwd=timing_folder, shell=True)

# abrimos el archivo generado por tempo2, donde se encuentra el RMS que buscamos

    rms[i] = np.genfromtxt ( timing folder + residue_file, comments="none", dtype=float, skip_header=18, max_rows=1, usecols=(10) )

    print(str(nbin_array[i]) + ' ' + str(rms[i] )
          
    i += 1

In [ ]:
# Set the fonts. Always use big fonts.
nice_fonts = {
        # Use LaTeX to write all text
        "text.usetex": True,
        "font.family": "serif",
        # Use 10pt font in plots, to match 10pt font in document
        "axes.labelsize": 30,
        "font.size": 28,
        "axes.linewidth": 1.5,
        # Make the legend/label fonts a little smaller
        "legend.fontsize": 32,
        "xtick.labelsize": 26,
        "ytick.labelsize": 26,
}
# Update the fonts
mpl.rcParams.update(nice_fonts)
#plt.tight_layout()

fig, (ax) = plt.subplots(1, 1, figsize=(11,9))

ax.set(xlabel='${\mathrm{n_{bins}}}$', ylabel="$ {\mathrm{RMS}}~[\mu s] $")
ax.grid()

ax.set_xscale('log', basex=2)

ax.get_xaxis().set_major_formatter(mpl.ticker.FuncFormatter(lambda x, p: format(int(x), '')))

plt.setp(ax1, xticks=[64,128,256,512,1024])

# buscamos los valores de chisq_red que se corresponden a una confianza de 1 sigma

prob_84 = scipy.stats.chi2.ppf( 0.84, n_free)
prob_16 = scipy.stats.chi2.ppf( 0.16, n_free)

# barremos todos los conjuntos de nbins

rms_max = np.zeros(5)
rms_min = np.zeros(5)

for x in range(0,5):
    
   # calculamos las barras de error para A1

    rms_max[x] = round(math.sqrt( ((n_free) / prob_16 * rms[x] , 4)
    rms_min[x] = round(math.sqrt( ((n_free) / prob_84 * rms[x] , 4)


asymmetric_error = [abs(rms_min_A1-rmsA1), abs(rms_max_A1-rmsA1)]

ax.scatter(nbin_array, rms, label='', marker="s", lw=7)
ax.errorbar(nbin_array, rms, yerr=asymmetric_error, label='A1', marker="s", fmt='o', capthick=3, capsize=5)
ax.legend(loc='upper right')

#fig.tight_layout()
plt.savefig('nbins_rms_' + ant + '.pdf', bbox_inches='tight')
plt.show()